# Internet Archive TV news analysis <a class="tocSkip">
This document contains the code and corresponding visualizations/statistics for answering various questions about the TV news dataset.

All times shown are H*:MM:SS.

In [ ]:
%matplotlib inline
from query.datasets.prelude import *
import query.datasets.tvnews.queries as queries
from query.datasets.tvnews.validation import *
import pyspark.sql.functions as func
import IPython
import shutil

rudecarnie = Labeler.objects.get(name='rudecarnie')
mtcnn = Labeler.objects.get(name='mtcnn')

spark = SparkWrapper()

def format_time(seconds, padding=4):
    return '{{:0{}d}}:{{:02d}}:{{:02d}}'.format(padding).format(seconds/3600, seconds/60 % 60, seconds % 60)

def format_number(n):
    def fmt(n):
        suffixes = {
            6: 'thousand',
            9: 'million',
            12: 'billion',
            15: 'trillion'
        }

        log = math.log10(n)
        suffix = None
        key = None
        for k in sorted(suffixes.keys()):
            if log < k:
                suffix = suffixes[k]
                key = k
                break

        return '{:.2f} {}'.format(n / float(10**(key-3)), suffix)
    if isinstance(n, list):
        return map(fmt, n)
    else:
        return fmt(n)

def show_df(table, ordering, clear=True):
    if clear:
        IPython.display.clear_output()
    return pd.DataFrame(table)[ordering]
        
def format_hour(h):
    if h <= 12:
        return '{} AM'.format(h)
    else:
        return '{} PM'.format(h-12)
    
hours = [r['hour'] for r in 
         Video.objects.annotate(hour=Extract('time', 'hour')).distinct('hour').order_by('hour').values('hour')]

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#All-videos" data-toc-modified-id="All-videos-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>All videos</a></span></li><li><span><a href="#Videos-by-channel" data-toc-modified-id="Videos-by-channel-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Videos by channel</a></span></li><li><span><a href="#Videos-by-show" data-toc-modified-id="Videos-by-show-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Videos by show</a></span></li><li><span><a href="#Videos-by-time-of-day" data-toc-modified-id="Videos-by-time-of-day-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Videos by time of day</a></span></li></ul></li><li><span><a href="#Shots" data-toc-modified-id="Shots-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Shots</a></span><ul class="toc-item"><li><span><a href="#Shot-validation" data-toc-modified-id="Shot-validation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Shot validation</a></span></li><li><span><a href="#All-shots" data-toc-modified-id="All-shots-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>All shots</a></span></li><li><span><a href="#Shots-by-channel" data-toc-modified-id="Shots-by-channel-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Shots by channel</a></span></li><li><span><a href="#Shots-by-show" data-toc-modified-id="Shots-by-show-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Shots by show</a></span></li><li><span><a href="#Shots-by-time-of-day" data-toc-modified-id="Shots-by-time-of-day-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Shots by time of day</a></span></li></ul></li><li><span><a href="#Commercials" data-toc-modified-id="Commercials-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Commercials</a></span><ul class="toc-item"><li><span><a href="#All-commercials" data-toc-modified-id="All-commercials-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>All commercials</a></span></li><li><span><a href="#Commercials-by-channel" data-toc-modified-id="Commercials-by-channel-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Commercials by channel</a></span></li><li><span><a href="#Commercials-by-show" data-toc-modified-id="Commercials-by-show-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Commercials by show</a></span></li><li><span><a href="#Commercials-by-time-of-day" data-toc-modified-id="Commercials-by-time-of-day-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Commercials by time of day</a></span></li></ul></li><li><span><a href="#Faces" data-toc-modified-id="Faces-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Faces</a></span><ul class="toc-item"><li><span><a href="#Face-validation" data-toc-modified-id="Face-validation-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Face validation</a></span></li><li><span><a href="#All-faces" data-toc-modified-id="All-faces-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>All faces</a></span></li></ul></li><li><span><a href="#Gender" data-toc-modified-id="Gender-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Gender</a></span><ul class="toc-item"><li><span><a href="#Gender-validation" data-toc-modified-id="Gender-validation-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Gender validation</a></span></li><li><span><a href="#All-gender" data-toc-modified-id="All-gender-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>All gender</a></span></li><li><span><a href="#Gender-by-channel" data-toc-modified-id="Gender-by-channel-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Gender by channel</a></span></li><li><span><a href="#Gender-by-show" data-toc-modified-id="Gender-by-show-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Gender by show</a></span></li><li><span><a href="#Gender-by-time-of-day" data-toc-modified-id="Gender-by-time-of-day-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Gender by time of day</a></span></li><li><span><a href="#Gender-by-day-of-the-week" data-toc-modified-id="Gender-by-day-of-the-week-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Gender by day of the week</a></span></li><li><span><a href="#Gender-by-topic" data-toc-modified-id="Gender-by-topic-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>Gender by topic</a></span></li><li><span><a href="#Male-vs.-female-faces-in-panels" data-toc-modified-id="Male-vs.-female-faces-in-panels-5.8"><span class="toc-item-num">5.8&nbsp;&nbsp;</span>Male vs. female faces in panels</a></span></li></ul></li><li><span><a href="#Pose" data-toc-modified-id="Pose-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Pose</a></span></li><li><span><a href="#Topics" data-toc-modified-id="Topics-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Topics</a></span></li><li><span><a href="#Figures" data-toc-modified-id="Figures-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Figures</a></span></li></ul></div>

<hr />
# Dataset

In [ ]:
def load_videos():
    return spark.qs_to_df(
        Video.objects.all().annotate( 
            hour=Extract('time', 'hour'), duration=Cast(F('num_frames') / F('fps'), models.IntegerField())) \
        .values('path', 'num_frames', 'fps', 'show_id', 'channel_id', 'hour', 'duration'))
videos = spark.load('videos', load_videos)

In [ ]:
def video_stats(key, labels):
    if key is not None:
        rows = videos.groupBy(key).agg(
            videos[key], func.count('duration'), func.avg('duration'), func.sum('duration'), func.stddev_pop('duration')) \
            .collect()
    else:
        rows = videos.agg(
            func.count('duration'), func.avg('duration'), func.sum('duration'), func.stddev_pop('duration')).collect()
    rmap = {(0 if key is None else r[key]): r for r in rows}
    return [{
        'label': label['name'],
        'count': rmap[label['id']]['count(duration)'],
        'duration': format_time(int(rmap[label['id']]['sum(duration)'])),
        'avg_duration': '{} (σ = {})'.format(
            format_time(int(rmap[label['id']]['avg(duration)'])),
            format_time(int(rmap[label['id']]['stddev_pop(duration)']), padding=0))
    } for label in labels]

video_ordering = ['label', 'count', 'duration', 'avg_duration']

<hr />
## All videos

In [ ]:
show_df(
    video_stats(None, [{'id': 0, 'name': 'whole dataset'}]),
    video_ordering)

<hr />
## Videos by channel

In [ ]:
show_df(
    video_stats('channel_id', list(Channel.objects.all().values('id', 'name'))),
    video_ordering)

<hr />
## Videos by show
"Situation Room with Wolf Blitzer" and "Special Report with Bret Baier" were ingested as 60 10-minute segments each, whereas the other shows have 10 ≥1 hour segments.

In [ ]:
show_df(
    video_stats('show_id', list(Show.objects.all().values('id', 'name'))),
    video_ordering)

<hr />
## Videos by time of day
Initial selection of videos was only prime-time, so between 4pm-11pm.

In [ ]:
show_df(
    video_stats('hour', [{'id': hour, 'name': format_hour(hour)} for hour in hours]),
    video_ordering)

# Shots

In [ ]:
def track_fields(qs, more_fields=[]):
    return qs.annotate( 
            hour=Extract('video__time', 'hour'), 
            week_day=Extract('video__time', 'week_day'),
            duration=Cast(
                (F('max_frame') - F('min_frame')) / F('video__fps'),
                models.FloatField())) \
        .values(*(['id', 'min_frame', 'max_frame', 'video__channel', 'video__show', 'duration', 'hour', 
                   'video_id', 'week_day', 'video__time'] + more_fields)) \
        .order_by('id')

shot_labeler = Labeler.objects.get(name='shot-histogram')
def load_shots():
    return spark.qs_to_df(track_fields(Shot.objects.filter(labeler=shot_labeler)))
shots = spark.load('shots', load_shots)

commercial_labeler = Labeler.objects.get(name='haotian-commercials')
def load_commercials():
    return spark.qs_to_df(track_fields(Commercial.objects.filter(labeler=commercial_labeler)))
commercials = spark.load('commercials', load_commercials)

segment_labeler = Labeler.objects.get(name='haotian-segments')
def load_segments():
    return spark.qs_to_df(
        track_fields(Segment.objects.filter(labeler=segment_labeler), ['polarity', 'subjectivity']))
segments = spark.load('segments', load_segments)

def match_segments(df):
    with Timer('collect'):
        fields = ['id', 'min_frame', 'max_frame', 'video_id']
        shots_list = df.select(*fields).collect()
        commercials_list = commercials.select(*fields).collect()
        segments_list = segments.select(*fields).collect()

    with Timer('group by key'):
        grouped_shots = collect(shots_list, itemgetter('video_id'))
        grouped_commercials = collect(commercials_list, itemgetter('video_id'))
        grouped_segments = collect(segments_list, itemgetter('video_id'))

    def inrange(a, b):
        return b['min_frame'] <= a['min_frame'] and a['max_frame'] <= b['max_frame']

    in_commercial_dict = {d['id']: False for d in tqdm(shots_list)}
    segment_col = []
    for video_id, vid_shots in tqdm(grouped_shots.iteritems()):
        if video_id not in grouped_commercials: continue
        vid_commercials = grouped_commercials[video_id]
        vid_segments = grouped_segments[video_id]

        for shot in vid_shots:
            segment_id = None
            for commercial in vid_commercials:
                if inrange(shot, commercial):
                    in_commercial_dict[shot['id']] = True
                    break
            for segment in vid_segments:
                if inrange(shot, segment):
                    segment_id = segment['id']
                    break
            segment_col.append([shot['id'], segment_id])

    sorted_col = [[k, in_commercial_dict[k]] for k in tqdm(sorted(in_commercial_dict.keys()))]    
    df1 = spark.append_column(df, 'in_commercial', sorted_col)
    return spark.append_column(df1, 'segment_id', sorted(segment_col, key=itemgetter(0)))
     
def load_shots2():
    return match_segments(shots)

shots2 = spark.load('shots2', load_shots2)
shots = shots2.where(shots2.in_commercial == False)
shots_com = shots2.where(shots2.in_commercial == True)

speaker_labeler, _ = Labeler.objects.get_or_create(name='lium')
def load_speakers():
    return spark.qs_to_df(track_fields(Speaker.objects.filter(labeler=speaker_labeler), ['gender_id']))
speakers = spark.load('speakers', load_speakers)

def load_speakers2():
    return match_segments(speakers)
speakers2 = spark.load('speakers2', load_speakers2)
speakers2 = speakers2.where(speakers2.in_commercial == False)

def load_faces():
    return spark.qs_to_df(Face.objects \
        .annotate(height=F('bbox_y2') - F('bbox_y1')) \
        .filter(labeler=mtcnn) \
        .annotate(
            duration=Cast(
                (F('shot__max_frame') - F('shot__min_frame')) / F('shot__video__fps'),
                models.FloatField()),
            hour=Extract('person__frame__video__time', 'hour')) \
        .values('duration', 'person__frame__video__channel', 'person__frame__video__show', 'hour', 'shot', 'is_host'))

faces = spark.load('faces', load_faces)
faces_cols = faces.columns
faces = faces.join(shots, faces.shot_id == shots.id).select(*[faces[k] for k in faces_cols])

def filter_hosts():
    with Timer('collect'):
        fields = ['id', 'min_frame', 'max_frame', 'video_id']
        speakers_list = speakers2.select(*fields).collect()
        hosts = faces.where(faces.is_host == True)
        shots_list = shots.join(hosts, shots.id == faces.shot_id, 'inner').select(*fields).collect()

    with Timer('group by key'):
        grouped_shots = collect(shots_list, itemgetter('video_id'))
        grouped_speakers = collect(speakers_list, itemgetter('video_id'))

    def inrange(a, b):
        return b['min_frame'] <= a['min_frame'] and a['max_frame'] <= b['max_frame']

    has_host_dict = {d['id']: False for d in tqdm(speakers_list)}
    for video_id, vid_speakers in tqdm(list(grouped_speakers.iteritems())):
        if video_id not in grouped_shots: continue
        vid_shots = grouped_shots[video_id]

        for speaker in vid_speakers:
            for shot in vid_shots:
                if inrange(speaker, shot):
                    has_host_dict[speaker['id']] = True
                    break

    sorted_col = [[k, has_host_dict[k]] for k in tqdm(sorted(has_host_dict.keys()))]    
    df1 = spark.append_column(speakers2, 'has_host', sorted_col)
    return df1
speakers = spark.load('speakers3', filter_hosts)

def load_segment_links():
    return spark.qs_to_df(
        Segment.things.through.objects.filter(tvnews_segment__labeler=segment_labeler) \
        .values('id', 'tvnews_segment_id', 'tvnews_thing_id').order_by('id'))
segment_links = spark.load('segment_links', load_segment_links).withColumnRenamed('tvnews_segment_id', 'segment_id') \
    .withColumnRenamed('tvnews_thing_id', 'thing_id')

def load_things():
    return spark.qs_to_df(Thing.objects.values('id', 'name', 'type').order_by('id'))
things = spark.load('things', load_things)

In [ ]:
med_withcom = shots_com.approxQuantile('duration', [0.5], 0.01)[0]
med_nocom = shots.approxQuantile('duration', [0.5], 0.01)[0]
med_channels = {
    c.name: shots.where(shots.channel_id == c.id).approxQuantile('duration', [0.5], 0.01)[0]
    for c in Channel.objects.all()
}
pickle.dump({
    'withcom': med_withcom,
    'nocom': med_nocom,
    'channels': med_channels
}, open('/app/data/shot_medians.pkl', 'wb'))

In [ ]:
all_shot_durations =  np.array([r['duration'] for r in shots.select('duration').collect()])
hist, edges = np.histogram(all_shot_durations, bins=list(range(0, 3600)) + [10000000])

In [ ]:
pickle.dump(hist, open('/app/data/shot_histogram.pkl', 'wb'))

In [ ]:
# print(spark.median(shots.where(shots.channel_id == Channel.objects.get(name='FOXNEWS').id), 'duration'))
# print(spark.median(shots.where(shots.channel_id == Channel.objects.get(name='CNN').id), 'duration'))

In [ ]:
# import scipy
# import pylab
# #scipy.stats.skewtest(np.log10(durations2))
# scipy.stats.probplot(np.log10(durations2), dist="norm", plot=pylab)
# pylab.show()

In [ ]:
# def loaddur(df):
#     durations = df.select('duration').collect()
#     return [d.duration for d in tqdm(durations)]
# durations2 = loaddur(shots)
# #cnn_dur = loaddur(shots.where(shots.channel_id == Channel.objects.get(name='CNN').id))
# #fox_dur = loaddur(shots.where(shots.channel_id == Channel.objects.get(name='FOXNEWS').id))

In [ ]:
# N = 3599

# cnn_hist = np.histogram(durations2, bins=list(range(N)) + [3600])
# #fox_hist = np.histogram(cnn_dur, bins=list(range(N)) + [3600])

# plt.title('TV news shot duration histogram')
# plt.xlabel('Shot length')
# plt.ylabel('Frequency')
# plt.xscale('log')
# plt.yscale('log')
# plt.gca().yaxis.grid(True)
# plt.plot(list(range(N)), cnn_hist[0], 'r')
# #plt.plot(list(range(N)), fox_hist[0], 'g')
# plt.savefig('shot_duration_histogram_loglog.png', dpi=300)
# plt.tight_layout()

## Shot validation

In [ ]:
# From Sahaj
shot_precision = 0.97
shot_recall = 0.97  

def shot_error_interval(n):
    return [n * shot_precision, n * (2 - shot_recall)]

In [ ]:
def shot_stats(key, labels, shots=shots):
    if key is not None:
        df = shots.groupBy(key)
        rows = df.agg(shots[key], func.count('duration'), func.avg('duration'), func.sum('duration'), func.stddev_pop('duration')).collect()
    else:
        df = shots
        rows = df.agg(func.count('duration'), func.avg('duration'), func.sum('duration'), func.stddev_pop('duration')).collect()
    rmap = {(0 if key is None else r[key]): r for r in rows}
    out_rows = []
    for label in labels:
        try:
            out_rows.append({
                'label': label['name'],
                'count': format_number(shot_error_interval(rmap[label['id']]['count(duration)'])),
                'duration': format_time(int(rmap[label['id']]['sum(duration)'])),
                'avg_duration': '{:06.2f}s (σ = {:06.2f})'.format(
                    rmap[label['id']]['avg(duration)'],
                    rmap[label['id']]['stddev_pop(duration)'])
            })
        except KeyError:
            pass
    return out_rows

shot_ordering = ['label', 'count', 'duration', 'avg_duration']

## All shots

In [ ]:
show_df(
    shot_stats(None, [{'id': 0, 'name': 'whole dataset'}], shots=shots_com),
    shot_ordering)

## Shots by channel

In [ ]:
show_df(
    shot_stats('channel_id', list(Channel.objects.all().values('id', 'name'))),
    shot_ordering)

## Shots by show

In [ ]:
show_df(
    shot_stats('show_id', list(Show.objects.all().values('id', 'name'))),
    shot_ordering)

## Shots by time of day

In [ ]:
# TODO: multi hour videos are lumped into a single hour bin

show_df(
    shot_stats('hour', [{'id': hour, 'name': format_hour(hour)} for hour in hours]),
    shot_ordering)

# Commercials

In [ ]:
def commercial_stats(key, labels):
    if key is not None:
        rows = commercials.groupBy(key).agg(commercials[key], func.count('duration'), func.avg('duration'), func.sum('duration')).collect()
    else:
        rows = commercials.agg(func.count('duration'), func.avg('duration'), func.sum('duration')).collect()
    rmap = {(0 if key is None else r[key]): r for r in rows}
    out_rows = []
    for label in labels:
        try:
            out_rows.append({
                'label': label['name'],
                'count': format_number(rmap[label['id']]['count(duration)']),
                'duration': format_time(int(rmap[label['id']]['sum(duration)'])),
                'avg_duration': '{:06.2f}s'.format(rmap[label['id']]['avg(duration)'])
            })
        except KeyError:
            pass
    return out_rows

commercial_ordering = ['label', 'count', 'duration', 'avg_duration']

## All commercials

In [ ]:
# TODO: compute avg # commercials per video
show_df(
    commercial_stats(None, [{'id': 0, 'name': 'whole dataset'}]),
    commercial_ordering)

## Commercials by channel

In [ ]:
show_df(
    commercial_stats('channel_id', list(Channel.objects.all().values('id', 'name'))),
    commercial_ordering)

## Commercials by show

In [ ]:
show_df(
    commercial_stats('show_id', list(Show.objects.all().values('id', 'name'))),
    commercial_ordering)

## Commercials by time of day

In [ ]:
show_df(
    commercial_stats('hour', [{'id': hour, 'name': format_hour(hour)} for hour in hours]),
    commercial_ordering)

# Faces

## Face validation

In [ ]:
base_face_stats = face_validation('All faces', lambda x: x)
big_face_stats = face_validation(
    'Faces height > 0.2', lambda qs: qs.annotate(height=F('bbox_y2') - F('bbox_y1')).filter(height__gte=0.2))

In [ ]:
def face_error_interval(n, face_stats):
    (face_precision, face_recall) = face_stats
    return [n * shot_precision * face_precision, n * (2 - shot_recall) * (2 - face_recall)]

## All faces

In [ ]:
print('Total faces: {}'.format(
    format_number(face_error_interval(faces.count(), base_face_stats[2]))))

total_duration = videos.agg(func.sum('duration')).collect()[0]['sum(duration)'] - \
    commercials.agg(func.sum('duration')).collect()[0]['sum(duration)']
face_duration = faces.groupBy('shot_id') \
    .agg(func.first('duration').alias('duration')).agg(func.sum('duration')).collect()[0]['sum(duration)']
print('% of time a face is on screen: {}'.format(100.0 * face_duration / total_duration))

<hr />
# Gender
These queries analyze the distribution of men vs. women across a number of axes. We use faces detected by [MTCNN](https://github.com/kpzhang93/MTCNN_face_detection_alignment/) and gender detected by [rude-carnie](https://github.com/dpressel/rude-carnie). We only consider faces with a height > 20% of the frame to eliminate people in the background.

Time for a given gender is the amount of time during which at least one person of that gender was on screen. Percentages are (gender screen time) / (total time any person was on screen).


In [ ]:
def load_genders():
    return spark.qs_to_df(FaceGender.objects \
        .annotate(height=F('face__bbox_y2') - F('face__bbox_y1')) \
        .filter(labeler=rudecarnie, face__labeler=mtcnn) \
        .annotate(
            duration=Cast(
                (F('face__shot__max_frame') - F('face__shot__min_frame')) / F('face__shot__video__fps'),
                models.FloatField()),
            hour=Extract('face__person__frame__video__time', 'hour'),
            week_day=Extract('face__person__frame__video__time', 'week_day')) \
        .values('gender', 'height', 'duration', 'face__person__frame__video__channel', 'face__person__frame__video__show', 'hour', 'face__shot', 'week_day', 'face__is_host'))

genders = spark.load('genders', load_genders)
genders = genders.where(genders.height > 0.2)
genders_cols = genders.columns
genders = genders.join(shots, genders.shot_id == shots.id).select(*([genders[k] for k in genders_cols] + [shots.segment_id, shots.video_id]))


## Gender validation


In [ ]:
# gender_validation('All gender', base_face_stats)

In [ ]:
_, Cm = gender_validation('Gender w/ face height > 0.2', big_face_stats)

def P(y, yhat):
    d = {'M': 0, 'F': 1, 'U': 2}
    return float(Cm[d[y]][d[yhat]]) / sum([Cm[i][d[yhat]] for i in d.values()])

In [ ]:
P('M', 'U')

In [ ]:
# TODO: remove a host -- use face features to identify and remove rachel maddow from computation
# TODO: more discrete time zones ("sunday mornings", "prime time", "daytime", "late evening")
# TODO: by year
# TODO: specific dates, e.g. during the RNC

MALE = Gender.objects.get(name='M')
FEMALE = Gender.objects.get(name='F')
UNKNOWN = Gender.objects.get(name='U')
gender_names = {g.id: g.name for g in Gender.objects.all()}

def gender_singlecount_stats(key, labels, min_dur=None):
    if key == 'topic':        
        df1 = genders.join(segment_links, genders.segment_id == segment_links.segment_id)
        df2 = df1.join(things, segment_links.thing_id == things.id)
        topic_type = ThingType.objects.get(name='topic').id
        df3 = df2.where(things.type_id == topic_type).select(
            *(['duration', 'channel_id', 'show_id', 'hour', 'week_day', 'gender_id'] +  \
              [things.id.alias('topic'), 'shot_id']))
        full_df = df3
    else:
        full_df = genders
        
    keys = ['duration', 'channel_id', 'show_id', 'hour', 'week_day']
    aggs = [func.count('gender_id')] + [func.first(full_df[k]).alias(k) for k in keys] + \
        ([full_df.topic] if key == 'topic' else [])
    groups = ([key] if key is not None else []) + ['gender_id']
    counts = full_df.groupBy(*(['shot_id', 'gender_id'] + (['topic'] if key == 'topic' else []))).agg(*aggs)
    rows = counts.where(counts['count(gender_id)'] > 0).groupBy(*groups) \
        .agg(func.sum('duration')).collect()

    if key is not None:
        base_counts = full_df.groupBy(['shot_id', key]).agg(full_df[key], func.first('duration').alias('duration')) \
            .groupBy(key).agg(full_df[key], func.sum('duration')).collect()
    else:
        base_counts = full_df.groupBy('shot_id').agg(func.first('duration').alias('duration')) \
            .agg(func.sum('duration')).collect()
    base_map = {
        (row[key] if key is not None else 0): row['sum(duration)']
        for row in base_counts
    }
        
    out_rows = []
    for label in labels:
        label_rows = {row.gender_id: row for row in rows if key is None or row[key] == label['id']}
        if len(label_rows) < 3: 
            continue

        base_dur = int(base_map[label['id']])
        if min_dur != None and base_dur < min_dur:
            continue
            
        durs = {
            g.id: int(label_rows[g.id]['sum(duration)'])
            for g in [MALE, FEMALE, UNKNOWN]
        }       
        
        def adjust(g):
            return int(reduce(lambda a, b: a + b, [durs[g2] * P(gender_names[g], gender_names[g2]) for g2 in durs]))
        
        adj_durs = {
            g: adjust(g)
            for g in durs
        }
            
        out_rows.append({
            key: label['name'],
            'M': format_time(durs[MALE.id]),
            'F': format_time(durs[FEMALE.id]),
            'U': format_time(durs[UNKNOWN.id]),
            'base': format_time(base_dur),
            'M%': int(100.0 * durs[MALE.id] / base_dur),
            'F%': int(100.0 * durs[FEMALE.id] / base_dur),
            'U%': int(100.0 * durs[UNKNOWN.id] / base_dur),
#             'M-Adj': format_time(adj_durs[MALE.id]),
#             'F-Adj': format_time(adj_durs[FEMALE.id]),
#             'U-Adj': format_time(adj_durs[UNKNOWN.id]),
#             'M-Adj%': int(100.0 * adj_durs[MALE.id] / base_dur),
#             'F-Adj%': int(100.0 * adj_durs[FEMALE.id] / base_dur),
#             'U-Adj%': int(100.0 * adj_durs[UNKNOWN.id] / base_dur),
            #'Overlap': int(100.0 * float(male_dur + female_dur) / base_dur) - 100
        })
    return out_rows
gender_ordering = ['M', 'M%', 'F', 'F%', 'U', 'U%']
#gender_ordering = ['M', 'M%', 'M-Adj', 'M-Adj%', 'F', 'F%', 'F-Adj', 'F-Adj%', 'U', 'U%', 'U-Adj', 'U-Adj%']

In [ ]:
def gender_multicount_stats(key, labels, min_dur=None, no_host=False, just_host=False):
    df0 = genders
    if no_host:
        df0 = df0.where(df0.is_host == False)        
    if just_host:
        df0 = df0.where(df0.is_host == True)
        
    if key == 'topic':        
        df1 = df0.join(segment_links, df0.segment_id == segment_links.segment_id)
        df2 = df1.join(things, segment_links.thing_id == things.id)
        topic_type = ThingType.objects.get(name='topic').id
        df3 = df2.where(things.type_id == topic_type).select(
            *(['duration', 'channel_id', 'show_id', 'hour', 'week_day', 'gender_id'] +  \
              [things.id.alias('topic'), 'shot_id']))
        full_df = df3
    else:
        full_df = df0
        
    groups = ([key] if key is not None else []) + ['gender_id']
    rows = full_df.groupBy(*groups).agg(func.sum('duration')).collect()
        
    out_rows = []
    for label in labels:
        label_rows = {row.gender_id: row for row in rows if key is None or row[key] == label['id']}
        if len(label_rows) < 3: continue
        male_dur = int(label_rows[MALE.id]['sum(duration)'])
        female_dur = int(label_rows[FEMALE.id]['sum(duration)'])
        unknown_dur = int(label_rows[UNKNOWN.id]['sum(duration)'])
        base_dur = male_dur + female_dur
        if min_dur != None and base_dur < min_dur:
            continue
        out_rows.append({
            key: label['name'],
            'M': format_time(male_dur),
            'F': format_time(female_dur),
            'U': format_time(unknown_dur),
            'base': format_time(base_dur),
            'M%': int(100.0 * male_dur / base_dur),
            'F%': int(100.0 * female_dur / base_dur),
            'U%': int(100.0 * unknown_dur / (base_dur + unknown_dur)),
            'Overlap': 0,
        })
    return out_rows

In [ ]:
def gender_speaker_stats(key, labels, min_dur=None, no_host=False):
    keys = ['duration', 'channel_id', 'show_id', 'hour', 'week_day']
    
    df0 = speakers
    if no_host:
        df0 = df0.where(df0.has_host == False)

    if key == 'topic':        
        df1 = df0.join(segment_links, speakers.segment_id == segment_links.segment_id)
        df2 = df1.join(things, segment_links.thing_id == things.id)
        topic_type = ThingType.objects.get(name='topic').id
        df3 = df2.where(things.type_id == topic_type).select(
            *(keys + ['gender_id', things.id.alias('topic')]))
        full_df = df3
    else:
        full_df = df0
  
    aggs = [func.count('gender_id')] + [func.first(full_df[k]).alias(k) for k in keys] + \
        ([full_df.topic] if key == 'topic' else [])
    groups = ([key] if key is not None else []) + ['gender_id'] + (['topic'] if key == 'topic' else [])
    rows = full_df.groupBy(*groups).agg(func.sum('duration')).collect()

    if key is not None:
        base_counts = full_df.groupBy(key).agg(full_df[key], func.sum('duration')).collect()
    else:
        base_counts = full_df.agg(func.sum('duration')).collect()
        
    base_map = {
        (row[key] if key is not None else 0): row['sum(duration)']
        for row in base_counts
    }
        
    out_rows = []
    for label in labels:
        label_rows = {row.gender_id: row for row in rows if key is None or row[key] == label['id']}
        if len(label_rows) < 2: continue
        male_dur = int(label_rows[MALE.id]['sum(duration)'])
        female_dur = int(label_rows[FEMALE.id]['sum(duration)'])
        base_dur = int(base_map[label['id']])
        if min_dur != None and base_dur < min_dur:
            continue
        out_rows.append({
            key: label['name'],
            'M': format_time(male_dur),
            'F': format_time(female_dur),
            'base': format_time(base_dur),
            'M%': int(100.0 * male_dur / base_dur),
            'F%': int(100.0 * female_dur / base_dur),
        })
    return out_rows

gender_speaker_ordering = ['M', 'M%', 'F', 'F%']

<hr />
## All gender


In [ ]:
print('Singlecount')
show_df(gender_singlecount_stats(None, [{'id': 0, 'name': 'whole dataset'}]), gender_ordering)

In [ ]:
print('Multicount')
gender_screen_all = gender_multicount_stats(None, [{'id': 0, 'name': 'whole dataset'}])
gender_screen_all_nh = gender_multicount_stats(None, [{'id': 0, 'name': 'whole dataset'}], no_host=True)
show_df(gender_screen_all, gender_ordering)

In [ ]:
show_df(gender_screen_all_nh, gender_ordering)

In [ ]:
print('Speaking time')
gender_speaking_all = gender_speaker_stats(None, [{'id': 0, 'name': 'whole dataset'}])
gender_speaking_all_nh = gender_speaker_stats(None, [{'id': 0, 'name': 'whole dataset'}], no_host=True)
show_df(gender_speaking_all, gender_speaker_ordering)

In [ ]:
show_df(gender_speaking_all_nh, gender_speaker_ordering)

<hr />
## Gender by channel


In [ ]:
print('Singlecount')
show_df(
    gender_singlecount_stats('channel_id', list(Channel.objects.values('id', 'name'))),
    ['channel_id'] + gender_ordering)

In [ ]:
print('Multicount')
show_df(
    gender_multicount_stats('channel_id', list(Channel.objects.values('id', 'name'))),
    ['channel_id'] + gender_ordering)

In [ ]:
print('Speaking time')
show_df(
    gender_speaker_stats('channel_id', list(Channel.objects.values('id', 'name'))),
    ['channel_id'] + gender_speaker_ordering)

<hr />
## Gender by show


In [ ]:
print('Singlecount')
show_df(
    gender_singlecount_stats('show_id', list(Show.objects.values('id', 'name')), min_dur=3600*500),
    ['show_id'] + gender_ordering)

In [ ]:
print('Multicount')
gender_screen_show = gender_multicount_stats('show_id', list(Show.objects.values('id', 'name')), min_dur=3600*250)
gender_screen_show_nh = gender_multicount_stats('show_id', list(Show.objects.values('id', 'name')), min_dur=3600*250, no_host=True)
gender_screen_show_jh = gender_multicount_stats('show_id', list(Show.objects.values('id', 'name')), min_dur=3600*50, just_host=True)
show_df(gender_screen_show, ['show_id'] + gender_ordering)


In [ ]:
#shots.show()
gshow = genders.groupBy('video_id', 'gender_id').agg(func.sum('duration').alias('screen_sum'), func.first('show_id').alias('show_id'))
gspeak = speakers.groupBy('video_id', 'gender_id').agg(func.sum('duration').alias('speak_sum'))
rows = gshow.join(gspeak, ['video_id', 'gender_id']).toPandas()

In [ ]:
show = Show.objects.get(name='Fox and Friends First')
rows2 = rows[rows.show_id == show.id]
videos = collect([r for _, r in rows2.iterrows()], lambda r: int(r.video_id))
bs = []
vkeys = []
for vid, vrows in videos.iteritems():
    vgender = {int(r.gender_id): r for r in vrows}
    def balance(key):
        return vgender[1][key] / float(vgender[1][key] + vgender[2][key])
    try:
        bs.append(balance('screen_sum') / balance('speak_sum'))
    except KeyError:
        bs.append(0)
    vkeys.append(vid)
idx = np.argsort(bs)[-20:]
print(np.array(vkeys)[idx].tolist(), np.array(bs)[idx].tolist())

In [ ]:
print(videos[14087])

In [ ]:
show_df(gender_screen_show_nh, ['show_id'] + gender_ordering)

In [ ]:
print('Speaking time')
gender_speaking_show = gender_speaker_stats('show_id', list(Show.objects.values('id', 'name')), min_dur=3600*3)
gender_speaking_show_nh = gender_speaker_stats('show_id', list(Show.objects.values('id', 'name')), min_dur=3600*3, no_host=True)
show_df(    
    gender_speaking_show,
    ['show_id'] + gender_speaker_ordering)

In [ ]:
show_df(    
    gender_speaking_show_nh,
    ['show_id'] + gender_speaker_ordering)

<hr />
## Gender by time of day

In [ ]:
print('Singlecount')
show_df(
    gender_singlecount_stats('hour', [{'id': hour, 'name': format_hour(hour)} for hour in hours]),
    ['hour'] + gender_ordering)   

In [ ]:
print('Multicount')
gender_screen_tod = gender_multicount_stats('hour', [{'id': hour, 'name': format_hour(hour)} for hour in hours])
show_df(gender_screen_tod, ['hour'] + gender_ordering)   

In [ ]:
print('Speaking time')
gender_speaking_tod = gender_speaker_stats('hour', [{'id': hour, 'name': format_hour(hour)} for hour in hours])
show_df(gender_speaking_tod, ['hour'] + gender_speaker_ordering)   

## Gender by day of the week

In [ ]:
dotw = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
print('Singlecount')
show_df(
    gender_singlecount_stats('week_day', [{'id': i+1, 'name': d} for i, d in enumerate(dotw)]),
    ['week_day'] + gender_ordering)

In [ ]:
print('Multicount')
show_df(
    gender_multicount_stats('week_day', [{'id': i+1, 'name': d} for i, d in enumerate(dotw)]),
    ['week_day'] + gender_ordering)

In [ ]:
print('Speaking time')
show_df(
    gender_speaker_stats('week_day', [{'id': i+1, 'name': d} for i, d in enumerate(dotw)]),
    ['week_day'] + gender_speaker_ordering)

<hr />
## Gender by topic

In [ ]:
# THOUGHTS:
# - Try topic analysis just on a "serious" news show. 
# - Generate a panel from multiple clips, e.g. endless panel of people on a topic
# - Produce an endless stream of men talking about, e.g. birth control

print('Singlecount')
show_df(
    gender_singlecount_stats(
        'topic', [{'id': t.id, 'name': t.name} for t in Thing.objects.filter(type__name='topic')],
        min_dur=3600*5),
    ['topic'] + gender_ordering)

# check this 
# M% is the pecent of time that men are on screen when this topic is being discussed

In [ ]:
print('Multicount')
gender_screen_topic = gender_multicount_stats(
        'topic', [{'id': t.id, 'name': t.name} for t in Thing.objects.filter(type__name='topic')],
        min_dur=3600*300)
gender_screen_topic_nh = gender_multicount_stats(
        'topic', [{'id': t.id, 'name': t.name} for t in Thing.objects.filter(type__name='topic')],
        min_dur=3600*300, no_host=True)
show_df(gender_screen_topic, ['topic'] + gender_ordering)

In [ ]:
print('Speaking time')
gender_speaking_topic = gender_speaker_stats(
        'topic', [{'id': t.id, 'name': t.name} for t in Thing.objects.filter(type__name='topic')],
        min_dur=3600*100)
gender_speaking_topic_nh = gender_speaker_stats(
        'topic', [{'id': t.id, 'name': t.name} for t in Thing.objects.filter(type__name='topic')],
        min_dur=3600*100, no_host=True)
show_df(gender_speaking_topic, ['topic'] + gender_speaker_ordering)

<hr />
## Male vs. female faces in panels
* Smaller percentage of women in panels relative to overall dataset.

In [ ]:
# # TODO: female-domainated situations?
# # TODO: slice this on # of people in the panel
# # TODO: small visualization that shows sample of segments
# # TODO: panels w/ majority male vs. majority female

# print('Computing panels')
# panels = queries.panels()
# print('Computing gender stats')
# frame_ids = [frame.id for (frame, _) in panels]
# counts = filter_gender(lambda qs: qs.filter(face__person__frame__id__in=frame_ids), lambda qs: qs)
# show_df([counts], ordering)

<hr />
# Pose
* Animatedness of people (specifically hosts)
    * e.g. Rachel Maddow vs. others
    * Pick 3-4 hours of a few specific hosts, compute dense poses and tracks
    * Devise acceleration metric
* More gesturing on heated exchanges?
* Sitting vs. standing
* Repeated gestures (debates vs. state of the union)
* Head/eye orientation (are people looking at each other?)
* Camera orientation (looking at someone from above/below)
* How much are the hosts facing each other
* Quantify aggressive body language

# Topics

In [ ]:
topics = segments.join(segment_links, segments.id == segment_links.segment_id)
topics = topics.join(things, things.id == topics.thing_id)

def topic_stats(group, labels, type_name, time=False):
    ty = ThingType.objects.get(name=type_name)
    topic_names = {t.id: t.name for t in Thing.objects.filter(type=ty)}
    df = topics.where(topics.type_id == ty.id)

    aggs = [func.sum('duration'), func.avg('polarity'), func.avg('subjectivity')]
    if group is not None:
        groups = ([func.year('time'), func.month('time')] if time else [])
        rows = df.groupBy(['thing_id', group] + groups).agg(
            *([func.first(group)] + ([func.min('time')] if time else []) + groups + aggs)).sort('thing_id').collect()

        def row(r):
            new_row = {
                'duration': r['sum(duration)'],
                'subjectivity': r['avg(subjectivity)'],
                'polarity': r['avg(polarity)'],
                'topic': topic_names[r['thing_id']],
                group: labels[r[group]]
            }
            if time:
                new_row['time'] = r['min(time)']
            return new_row
    

        return pd.DataFrame([row(r) for r in rows])

    else:
        groups = [func.year('time'), func.month('time')] if time else []
        rows = df.groupBy(['thing_id'] + groups).agg(
            *(([func.min('time')] if time else []) + groups + aggs)).sort('thing_id').collect()
        def row(r):
            d =  {
                'topic': topic_names[r.thing_id],
                'duration': r['sum(duration)']
            }
            if time:
                d['time'] = r['min(time)']
            return d
        return pd.DataFrame([row(r) for r in rows])


In [ ]:
topics.show()

In [ ]:
topic_all = topic_stats(None, None, 'topic', time=True)

In [ ]:
topic_all[topic_all.topic == 'conservatives'].sort_values(by=['time']).plot('time', 'duration')

In [ ]:
dfs = topic_stats('channel_id', {c.id: c.name for c in Channel.objects.all()}, 'topic')
dfs['duration'].to_csv('/app/data/topic_channel.csv')

In [ ]:
dfs = topic_stats('channel_id', {c.id: c.name for c in Channel.objects.all()}, 'topic', time=True)

In [ ]:
#topics = dfs.topic.unique()
topics = 'doctors, poverty, email, privacy, electricity, refugees, christmas, savings, volcanoes, wiretapping, shopping, anxiety, inauguration'.split(', ')
def plt_topic(i, t):
    figure = plt.figure()
    ax = figure.add_subplot(1, 1, 1)
    ax.set_title(t)
    def plt_chan(c):
        dfs2 = dfs[dfs.channel_id == c]
        dfs2 = dfs2[dfs2.topic == t]
        if len(dfs2) > 0:
            dfs2.sort_values(by=['time']).plot('time', 'duration', ax=ax, label=c)
    plt_chan('CNN')
    plt_chan('FOXNEWS')    
    plt_chan('MSNBC')
    
# doctors, poverty, email, privacy, electricity, refugees, christmas, savings, volcanoes, wiretapping, shopping
# anxiety, inauguration
    
for i, t in tqdm(list(enumerate(topics))):
    plt_topic(i+1, t)
plt.tight_layout()

In [ ]:
dfs = topic_stats('show_id', {c.id: c.name for c in Show.objects.all()}, 'topic')
dfs['duration'].to_csv('/app/data/topic_show.csv')

In [ ]:
person_all = pd.DataFrame(topic_stats(None, None, 'person'))

In [ ]:
dfs = topic_stats('channel_id', {c.id: c.name for c in Channel.objects.all()}, 'person')
dfs['duration'].to_csv('/app/data/person_channel.csv')

In [ ]:
dfs = topic_stats('show_id', {c.id: c.name for c in Show.objects.all()}, 'person')
dfs['duration'].to_csv('/app/data/person_show.csv')

In [ ]:
dfs = topic_stats('channel_id', {c.id: c.name for c in Channel.objects.all()}, 'phrase')
dfs['duration'].to_csv('/app/data/phrase_channel.csv')

In [ ]:
dfs = topic_stats('show_id', {c.id: c.name for c in Show.objects.all()}, 'phrase')
dfs['duration'].to_csv('/app/data/phrase_show.csv')

<hr />
# Figures

In [ ]:
df = pd.DataFrame(gender_screen_tod)
ax = df.plot('hour', 'M%')
pd.DataFrame(gender_speaking_tod).plot('hour', 'M%', ax=ax)
ax.set_ylim(0, 100)
ax.set_xticks(range(len(df)))
ax.set_xticklabels(df.hour)
ax.axhline(50, color='r', linestyle='--')
ax.legend(['Screen time', 'Speaking time',  '50%'])

In [ ]:
pd.DataFrame(gender_screen_topic).to_csv('/app/data/screen_topic.csv')
pd.DataFrame(gender_screen_topic_nh).to_csv('/app/data/screen_topic_nh.csv')
pd.DataFrame(gender_speaking_topic).to_csv('/app/data/speaking_topic.csv')
pd.DataFrame(gender_speaking_topic_nh).to_csv('/app/data/speaking_topic_nh.csv')

pd.DataFrame(gender_screen_show).to_csv('/app/data/screen_show.csv')
pd.DataFrame(gender_screen_show_nh).to_csv('/app/data/screen_show_nh.csv')
pd.DataFrame(gender_screen_show_jh).to_csv('/app/data/screen_show_jh.csv')
pd.DataFrame(gender_speaking_show).to_csv('/app/data/speaking_show.csv')
pd.DataFrame(gender_speaking_show_nh).to_csv('/app/data/speaking_show_nh.csv')

pd.DataFrame(gender_screen_all).to_csv('/app/data/screen_all.csv')
pd.DataFrame(gender_screen_all_nh).to_csv('/app/data/screen_all_nh.csv')
pd.DataFrame(gender_speaking_all).to_csv('/app/data/speaking_all.csv')